#Alternative Data Credit Scoring Model

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


ValueError: ignored

In [7]:
os.listdir('/content')

['.config', 'sample_data']

In [0]:
import os
import numpy as np
import pandas as pd
from pandas._libs.interval import Interval
from sklearn.linear_model import LogisticRegression

This credit scoring model is built on the alternative data. Over 2.5 billion peoples dont have bank accounts and credit history, so traditional approach to calculate the credit score is impossible. But, there is a alternative way to calculate the credit score. We can collect the data from this peoples which is not related to the credit or financial data, and feed it to this model to get a credit score. This alternative data could be anything related to this peoples like married status, childrens, education, job, income, relatives, phone call history, SMS history, digital payment history, neighbors, home address, office address, assets, health, and many more

FEATURES :
1. Age (Integer) 
2. Marital Status (Bool)
       1 - Married
       0 - Not Married
3. Dependents (Integer) - Number of dependents (0, 1, 2, 3, 4, 5+)
4. Home Address (Coordinates)
5. Work Address (Coordinates)
6. Total years work experience - Number of work experienc (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10+)
7. Working Status (Unemployed => 0, Student => 1, Part-Time => 2, Full-Time => 3)
8. Highest education qualification  (High School => 0, Bachelor's  => 1, Master's => 2, Doctorate => 3)
9. Area of study (Life Sciences => 0, Mathematical Sciences => 1, Humanities => 2, Social Science => 3, Business => 4, Law => 5, Engineering & Technology => 6, Sport => 7)
10. Annual Income


Target -

  1 => Good User
  
  2 => Bad User

In [3]:
#Generating random alternative data for credit scoing model

rand_age_data = np.random.random_integers(21, 90, 1000)
rand_maritalstatus_data = np.random.random_integers(0, 1, 1000)
rand_dependents_data = np.random.random_integers(0, 20, 1000)
rand_workexp_data = np.random.random_integers(0, 30, 1000)
rand_working_status_data = np.random.random_integers(0, 3, 1000)
rand_edu_quali_data = np.random.random_integers(0, 3, 1000)
rand_area_study_data = np.random.random_integers(0, 7, 1000)
rand_annualinc_data = np.random.random_integers(0, 30, 1000)


rand_target_data = np.random.random_integers(0, 1, 1000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated. Please call randint(21, 90 + 1) instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: This function is deprecated. Please call randint(0, 20 + 1) instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: This function is deprecated. Please call randint(0, 30 + 1) instead
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: This function is deprec

In [0]:
df_orig = pd.DataFrame({'age':rand_age_data,
                        'marital_status': rand_maritalstatus_data,
                        'dependents' : rand_dependents_data,
                        'work_exp': rand_workexp_data,
                        'work_status': rand_working_status_data,
                        'education_quali': rand_edu_quali_data, 
                        'area_study': rand_area_study_data,
                        'annual_income': rand_annualinc_data,
                        'target' : rand_target_data})
df_orig.head()

,age,marital_status,dependents,work_exp,work_status,education_quali,area_study,annual_income,target
0,31,0,3,2,1,1,0,23,1
1,67,1,9,5,1,1,7,10,1
2,55,0,6,0,1,2,3,30,1
3,72,1,8,0,2,3,1,20,0
4,90,1,7,2,3,3,6,5,1


##Data Transformations

Transforming the data in a manner suited to our model.

In [0]:
# Continous values from original dataframe are replced with the ranges they lie in.
df_ranges = pd.DataFrame()

df_ranges['age_range'] = pd.cut(df_orig.age, [0, 25, 30, 35, 42, 51, 91, np.inf], right = False)
df_ranges['marital_status'] = df_orig.marital_status
df_ranges['dependents_range'] = pd.cut(df_orig.dependents, [0, 2, 6, np.inf], right = False)
df_ranges['work_exp_range'] = pd.cut(df_orig.work_exp, [0, 3, 6, 11, np.inf], right = False)
df_ranges['work_status'] = df_orig.work_status
df_ranges['education_quali'] = df_orig.education_quali
df_ranges['area_study'] = df_orig.area_study
df_ranges['annual_income_range'] = pd.cut(df_orig.annual_income, [0, 6, 11, 16, 21, 26, 31], right = False)
df_ranges['target'] = df_orig.target

df_ranges.head()

,age_range,marital_status,dependents_range,work_exp_range,work_status,education_quali,area_study,annual_income_range,target
0,"[30.0, 35.0)",0,"[2.0, 6.0)","[0.0, 3.0)",1,1,0,"[21, 26)",1
1,"[51.0, 91.0)",1,"[6.0, inf)","[3.0, 6.0)",1,1,7,"[6, 11)",1
2,"[51.0, 91.0)",0,"[6.0, inf)","[0.0, 3.0)",1,2,3,"[26, 31)",1
3,"[51.0, 91.0)",1,"[6.0, inf)","[0.0, 3.0)",2,3,1,"[16, 21)",0
4,"[51.0, 91.0)",1,"[6.0, inf)","[0.0, 3.0)",3,3,6,"[0, 6)",1


### Weight Of Evidence (woe) calculation

In [4]:
import os
os.listdir('/content')

['.config', 'sample_data']

![woe-formula]()

In [0]:
# 'df_dict' contains dataframes having woe values of every featured
df_dict = {}

# Iterating over names of columns of "df_ranges" dataframe
for name in df_ranges.columns:
  if name != 'target':
    
    # Getting the unique values of each column of "df_ranges" per interation
    try:
      unique_ranges = list(df_ranges[name].unique().categories) # If the dtype is "Interval" 
    except:
      unique_ranges = list(df_ranges[name].unique()) 

    len_df = len(unique_ranges)
    num_of_goods = 0
    num_of_bads = 0
    df = pd.DataFrame({('unique_' + name) : unique_ranges, # Column of unique values (groups)
                       '#B' : np.zeros(len_df),            # Number of bads in each group
                       '#G' : np.zeros(len_df),            # Number of goods in each group
                       '%B' : np.zeros(len_df),            # Percentage of bads with respect to all the group
                       '%G' : np.zeros(len_df),            # Percentage of goods with respect to all the group
                       '%G/%B' : np.zeros(len_df),         # Division of % of goods to % of bads of each group   
                       'woe' : np.zeros(len_df)})          # Natural log of the divided value in the previous column (Weight Of Evidence)

    #Calculating number of goods and bads in each groups and storing them to the dataframes
    for i, interval in enumerate(df['unique_' + name]):
      value_counts = df_ranges[df_ranges[name] == interval].target.value_counts()
      df.iloc[i, 1] = value_counts[0]
      df.iloc[i, 2] = value_counts[1]

    #Calculating the total number of bads and goods in "#B" and "#G" columns
    for num in df['#B']:
      num_of_bads += num

    for num in df['#G']:
      num_of_goods += num

    #Calculating % of bads and goods, dividing them and taking a natural log
    for i in range(len(df)):
      df.iloc[i, 3] = (df.iloc[i, 1] / num_of_bads) * 100
      df.iloc[i, 4] = (df.iloc[i, 2] / num_of_goods) * 100
      df.iloc[i, 5] = df.iloc[i, 4] / df.iloc[i, 3]
      df.iloc[i, 6] = np.log(df.iloc[i, 5])

    #Saving dataframes in each interation
    df_dict['df_' + name] = df

Printing each dataframes in the dictionary

In [0]:
df_dict['df_age_range']

,unique_age_range,#B,#G,%B,%G,%G/%B,woe
0,"[0.0, 25.0)",26.0,35.0,5.273834,6.903353,1.308982,0.269250
1,"[25.0, 30.0)",36.0,36.0,7.302231,7.100592,0.972387,-0.028002
2,"[30.0, 35.0)",39.0,38.0,7.910751,7.495069,0.947454,-0.053977
3,"[35.0, 42.0)",42.0,42.0,8.519270,8.284024,0.972387,-0.028002
4,"[42.0, 51.0)",53.0,57.0,10.750507,11.242604,1.045774,0.044758
5,"[51.0, 91.0)",297.0,299.0,60.243408,58.974359,0.978935,-0.021290


In [0]:
df_dict['df_marital_status']

,unique_marital_status,#B,#G,%B,%G,%G/%B,woe
0,0,236.0,261.0,47.870183,51.47929,1.075394,0.072687
1,1,257.0,246.0,52.129817,48.52071,0.930767,-0.071746


In [0]:
df_dict['df_dependents_range']

,unique_dependents_range,#B,#G,%B,%G,%G/%B,woe
0,"[0.0, 2.0)",102.0,93.0,20.689655,18.343195,0.886588,-0.120375
1,"[2.0, 6.0)",165.0,190.0,33.468560,37.475345,1.119718,0.113077
2,"[6.0, inf)",226.0,224.0,45.841785,44.181460,0.963781,-0.036891


In [0]:
df_dict['df_work_exp_range']

,unique_work_exp_range,#B,#G,%B,%G,%G/%B,woe
0,"[0.0, 3.0)",252.0,252.0,51.115619,49.704142,0.972387,-0.028002
1,"[3.0, 6.0)",241.0,255.0,48.884381,50.295858,1.028874,0.028465


In [0]:
df_dict['df_work_status']

,unique_work_status,#B,#G,%B,%G,%G/%B,woe
0,1,151.0,127.0,30.628803,25.049310,0.817835,-0.201095
1,2,121.0,124.0,24.543611,24.457594,0.996495,-0.003511
2,3,105.0,117.0,21.298174,23.076923,1.083516,0.080212
3,0,116.0,139.0,23.529412,27.416174,1.165187,0.152882


In [0]:
df_dict['df_education_quali']

,unique_education_quali,#B,#G,%B,%G,%G/%B,woe
0,1,122.0,117.0,24.746450,23.076923,0.932535,-0.069849
1,2,130.0,133.0,26.369168,26.232742,0.994826,-0.005187
2,3,107.0,120.0,21.703854,23.668639,1.090527,0.086661
3,0,134.0,137.0,27.180527,27.021696,0.994156,-0.005861


In [0]:
df_dict['df_area_study']

,unique_area_study,#B,#G,%B,%G,%G/%B,woe
0,0,57.0,59.0,11.561866,11.637081,1.006505,0.006484
1,7,60.0,63.0,12.170385,12.426036,1.021006,0.020788
2,3,59.0,57.0,11.967546,11.242604,0.939424,-0.062488
3,1,68.0,58.0,13.793103,11.439842,0.829389,-0.187067
4,6,63.0,63.0,12.778905,12.426036,0.972387,-0.028002
5,4,65.0,69.0,13.184584,13.609467,1.032226,0.031717
6,5,67.0,80.0,13.590264,15.779093,1.161059,0.149332
7,2,54.0,58.0,10.953347,11.439842,1.044415,0.043457


In [0]:
df_dict['df_annual_income_range']

,unique_annual_income_range,#B,#G,%B,%G,%G/%B,woe
0,"[0, 6)",108.0,102.0,21.906694,20.118343,0.918365,-0.085160
1,"[6, 11)",75.0,67.0,15.212982,13.214990,0.868665,-0.140797
2,"[11, 16)",76.0,89.0,15.415822,17.554241,1.138716,0.129901
3,"[16, 21)",76.0,85.0,15.415822,16.765286,1.087538,0.083916
4,"[21, 26)",80.0,76.0,16.227181,14.990138,0.923767,-0.079295
5,"[26, 31)",78.0,88.0,15.821501,17.357002,1.097052,0.092626


In [0]:
#Function that check whether a value lie in a interval or not
def contains(interval, value):
  """For Left closed Right open interval"""
  if value >= interval.left and value < interval.right:
    return True
  else:
    return False

In [0]:
#Function that return woe of a value 
def generate_woe(column, value):
  key = 'df_' + column
  try:
    df = df_dict[key]
    
    for i, col_val in enumerate(df.iloc[:, 0]):
      if value == col_val:
        return df.iloc[i, 6]
  except:
    key = key = 'df_' + column + '_range'
    df = df_dict[key]

    for i, interval in enumerate(df.iloc[:, 0]):
      if contains(interval, value):
        return df.iloc[i, 6]

In [0]:
#Creating a final dataframe, replacing original values in "df_orig" with there woe values

final_df = pd.DataFrame().reindex_like(df_orig)
for i, column in enumerate(df_orig.columns):
  if column != 'target':
    for j in range(len(df_orig)):
      final_df.iloc[j, i] = generate_woe(column, df_orig.iloc[j, i])

final_df.target = df_orig.target

final_df.head()

,age,marital_status,dependents,work_exp,annual_income,target
0,0.024706,-0.014347,0.030012,0.077876,-0.064883,0
1,-0.251628,-0.014347,0.051322,0.077876,-0.064883,0
2,0.024706,-0.014347,0.051322,0.077876,-0.064883,1
3,0.024706,0.014831,0.051322,0.077876,0.090083,1
4,-0.093797,0.014831,0.051322,-0.074079,0.090083,1


In [0]:
X = final_df.drop('target', 1)
y = final_df.target

In [0]:
# Using Logistic

model = LogisticRegression()
model.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
model.coef_

array([[0.81355016, 0.03081816, 0.65500888, 0.59403352, 0.83046484]])

##Scorcard Scaling
It is used to get the score from the output of a predictive classifier.
Scaling increases the understandability of scorecard to the non-experts.